In [1]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import scipy

In [6]:
#create dataframe from csv
opioid_df = pd.read_csv("opioid2016.csv")

#preview data
opioid_df.info()
opioid_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131550 entries, 0 to 1131549
Data columns (total 11 columns):
NPI                                         1131550 non-null int64
NPPES Provider Last Name                    1131518 non-null object
NPPES Provider First Name                   1131518 non-null object
NPPES Provider ZIP Code                     1131497 non-null float64
NPPES Provider State                        1131550 non-null object
Specialty Description                       1131550 non-null object
Total Claim Count                           1131550 non-null int64
Opioid Claim Count                          812547 non-null float64
Opioid Prescribing Rate                     812547 non-null float64
Extended-Release Opioid Claims              1003036 non-null float64
Extended-Release Opioid Prescribing Rate    388564 non-null float64
dtypes: float64(5), int64(2), object(4)
memory usage: 95.0+ MB


,NPI,NPPES Provider ZIP Code,Total Claim Count,Opioid Claim Count,Opioid Prescribing Rate,Extended-Release Opioid Claims,Extended-Release Opioid Prescribing Rate
count,1.131550e+06,1.131497e+06,1.131550e+06,812547.000000,812547.000000,1.003036e+06,388564.000000
mean,1.499842e+09,4.806815e+04,1.308800e+03,96.173274,10.972242,5.367090e+00,2.668037
std,2.879109e+08,3.029548e+04,3.147395e+03,315.482373,17.424112,4.679540e+01,7.169746
min,1.003000e+09,0.000000e+00,1.100000e+01,0.000000,0.000000,0.000000e+00,0.000000
25%,1.245689e+09,2.120100e+04,5.400000e+01,0.000000,0.000000,0.000000e+00,0.000000
50%,1.497973e+09,4.503900e+04,2.000000e+02,19.000000,2.901024,0.000000e+00,0.000000
75%,1.740699e+09,7.657400e+04,1.000000e+03,69.000000,14.152168,0.000000e+00,0.000000
max,1.993000e+09,9.992900e+04,3.242090e+05,21772.000000,100.000000,5.548000e+03,100.000000


In [19]:
#clean up column titles
opioid_df.columns = opioid_df.columns.str.strip().str.lower().str.replace(' ', '_')

In [20]:
opioid_df.head()

,npi,nppes_provider_last_name,nppes_provider_first_name,nppes_provider_zip_code,nppes_provider_state,specialty_description,total_claim_count,opioid_claim_count,opioid_prescribing_rate,extended-release_opioid_claims,extended-release_opioid_prescribing_rate
0,1003000126,ENKESHAFI,ARDALAN,21502.0,MD,Internal Medicine,545,23.0,4.220183,NaN,NaN
1,1003000142,KHALIL,RASHID,43623.0,OH,Anesthesiology,1733,1004.0,57.934218,63.0,6.2749
2,1003000167,ESCOBAR,JULIO,89403.0,NV,Dentist,49,11.0,22.448980,0.0,0.0000
3,1003000282,BLAKEMORE,ROSIE,37243.0,TN,Nurse Practitioner,146,NaN,NaN,0.0,NaN
4,1003000407,GIRARDI,DAVID,15825.0,PA,Family Practice,2225,17.0,0.764045,0.0,0.0000


The dataset has about 1.1 million doctors, which a quick Google search confirms is the number of all doctors in America. Data is missing on opioid and extended-release opioid claims, because not all physicians prescribe opioids. Also, some 0s are NaNs are interchanged, which may explain why there is less data on extended-release prescribing rate than extended-release claims.

<h2> Who prescribes the most opioids? </h2>

In [35]:
opioid_df.sort_values(by='opioid_claim_count', ascending=False).head(10)

,npi,nppes_provider_last_name,nppes_provider_first_name,nppes_provider_zip_code,nppes_provider_state,specialty_description,total_claim_count,opioid_claim_count,opioid_prescribing_rate,extended-release_opioid_claims,extended-release_opioid_prescribing_rate
363670,1326019712,MURPHY,MARK,35601.0,AL,Pain Management,32039,21772.0,67.954680,5548.0,25.482271
850551,1750347605,MURCIANO,ENRIQUE,33143.0,FL,Anesthesiology,30232,19563.0,64.709579,1453.0,7.427286
366525,1326092610,PATEL,ALPESH,70809.0,LA,Interventional Pain Management,28962,18502.0,63.883710,3840.0,20.754513
364323,1326035163,GARLAPATI,BUTCHAIAH,72117.0,AR,Physical Medicine and Rehabilitation,29059,17753.0,61.092949,3668.0,20.661297
690563,1619004744,MCGEE,DARIUS,39440.0,MS,Nurse Practitioner,27228,16456.0,60.437785,974.0,5.918814
48157,1043276488,CHAO,RAUL,33143.0,FL,Interventional Pain Management,23556,16045.0,68.114281,636.0,3.963852
805483,1710085196,LEDFORD,MELANIE,40353.0,KY,Physical Medicine and Rehabilitation,25693,15859.0,61.724983,1687.0,10.637493
1086409,1962405035,MARTUCCI,MARTIN,74104.0,OK,Pain Management,20052,15527.0,77.433672,3254.0,20.957043
401355,1356427850,KABAKIBOU,KAMAL,30327.0,GA,Pain Management,26989,15397.0,57.049168,3225.0,20.945639
81433,1073555397,REHEEM,MEDHAT,34613.0,FL,Interventional Pain Management,22086,15285.0,69.206737,3564.0,23.316977


Who are the biggest prescribers outside of Pain and Physical Medicine specialties?

In [54]:
opioid_df[~opioid_df['specialty_description'].str.contains("Pain|Rehabilitation")].sort_values(by='opioid_claim_count', ascending=False).head(10)

,npi,nppes_provider_last_name,nppes_provider_first_name,nppes_provider_zip_code,nppes_provider_state,specialty_description,total_claim_count,opioid_claim_count,opioid_prescribing_rate,extended-release_opioid_claims,extended-release_opioid_prescribing_rate
850551,1750347605,MURCIANO,ENRIQUE,33143.0,FL,Anesthesiology,30232,19563.0,64.709579,1453.0,7.427286
690563,1619004744,MCGEE,DARIUS,39440.0,MS,Nurse Practitioner,27228,16456.0,60.437785,974.0,5.918814
347380,1306853510,MOREE,LAMAR,31701.0,GA,Anesthesiology,28119,14092.0,50.115580,1810.0,12.844167
1040585,1912996059,GANTT,RODDIE,35801.0,AL,Anesthesiology,20295,13698.0,67.494457,1553.0,11.337422
1098104,1972509974,BAKER,JAY,65803.0,MO,Anesthesiology,16273,13581.0,83.457260,4400.0,32.398203
817232,1720060247,LACKEY,THOMAS,35901.0,AL,Anesthesiology,27538,12708.0,46.147142,1694.0,13.330186
1068093,1942332242,HARRIS,DANIEL,77702.0,TX,Anesthesiology,20465,12161.0,59.423406,2209.0,18.164625
963253,1851342372,LOVE,RILEY,42003.0,KY,Anesthesiology,16089,11891.0,73.907639,1472.0,12.379110
862930,1760472708,COLLINS,RONALD,35801.0,AL,Anesthesiology,17079,11056.0,64.734469,1197.0,10.826700
46933,1043245749,SUAREZ,EMILIO,33176.0,FL,Anesthesiology,18146,10911.0,60.128954,1610.0,14.755751


...Anesthesiologists